In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from dotenv import load_dotenv
import os
from transformers import AutoTokenizer, AutoModel
from sklearn.pipeline import Pipeline


load_dotenv()
DATASET_PATH = f'{os.getenv("DATASETS_PATH")}depression_ds.csv'
df = pd.read_csv(DATASET_PATH)




In [ ]:




df_train, df_test = train_test_split(df, test_size = 0.2, random_state=13)

In [7]:
from bert_transformer import BertTransformer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained("bert-base-uncased")

bert_transformer = BertTransformer(tokenizer, model)



In [7]:


classifier = svm.SVC(kernel='linear')

model = Pipeline([
        ("vectorizer", bert_transformer),
        ("classifier", classifier),
        ])

In [22]:
train_X = df_train['clean_text']
train_y = df_train['is_depression']

In [23]:
model.fit(train_X, train_y)

Pipeline(steps=[('vectorizer',
                 BertTransformer(bert_model=BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder...
                                 bert_tokenizer=PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}),
                                 embedding_func=<function BertTransformer.__init__.<locals>.<lambda> at 0x7f41a6306f80>)),
                ('classifier', SVC(kernel='linear'))])

In [25]:
test_X = df_test['clean_text']
test_y = df_test['is_depression']

print(len(test_X))

predictions = model.predict(test_X)

1547


In [35]:
failed = 0
for prediction, answer in zip(predictions, test_y):
  if prediction != answer:
    failed += 1

print(f"Succes rate: {100 - round(failed/1547*100, 2)} %")

Succes rate: 94.05 %
